# Modeling

## Reference

The code in this notebook is adapted and modified from the following Youtube tutorial: https://www.youtube.com/watch?v=doDUihpj6ro 

## Usage

In this notebook, an LSTM based neural network model can be trained with data generated by the notebook "1_Example-Data-Generation.ipynb"

The path to the input data `DATA_PATH` is set to 'MP_Data_test'. Change it, if you want to use different / your own data generated with "1_Example-Data-Generation.ipynb". 

The weights of the trained final model are saved with the name `model_name = 'first_model_whoop_whoop.h5'`. Change it, if you want ;)

Further, weights are saved in checkpoints every 100 epochs (change if needed). 

## 1. Install and Import Dependencies

### Install Dependencies

In [46]:
%pip install tensorflow-macos opencv-python mediapipe-silicon sklearn matplotlib
#!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe sklearn matplotlib # original code line from tutorial (he used a Windows system)

Note: you may need to restart the kernel to use updated packages.


### Import Dependencies

In [47]:
# general 
import numpy as np
import pandas as pd
import os # easier file path handling

# for device camera feed
import cv2 # opencv
from matplotlib import pyplot as plt # imshow for easy visualization
import time # to insert breaks / "sleep" in between frames
import mediapipe as mp # for accessing and reading from device camera

# for data pre-processing
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# for model building and training
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

# for model evaluation
from sklearn.metrics import multilabel_confusion_matrix, ConfusionMatrixDisplay, accuracy_score

## 2. Preprocess Data and Create Labels and Features

### Some global setup (edit to your needs)

In [48]:
# path for input data
#DATA_PATH = os.path.join('MP_Data')
DATA_PATH = os.path.join('../data/')

# X and y filenames
X_fname = 'X-data.npy'
y_fname = 'y-data.npy'

# name for saving the trained model
model_name = 'Easter_model_10signs_j_test'

# directory to save tensorflow callbacks (logs)
log_dir = os.path.join('./Modeling_logs', model_name)

# for tensorboard callbacks
tb_callback = TensorBoard(log_dir=log_dir)

# for checkpoint callbacks
checkpoint_name = "cp-{epoch:04d}.ckpt"
checkpoint_path = os.path.join(log_dir, checkpoint_name)
cp_callback = tf.keras.callbacks.ModelCheckpoint(
   filepath=checkpoint_path, 
   verbose=1, 
   save_weights_only=True,
   # Save weights, every 100 epochs 
   save_freq='epoch', 
   period=100)

# actions to detect (get later from our .json file)
#actions = np.array(['hello', 'thanks', 'iloveyou']) # from tutorial
actions = np.array ( ['alligator', 
                      'radio', 
                      'moon', 
                      'sleep', 
                      'grandpa', 
                      'tiger', 
                      'pencil', 
                      'sleepy', 
                      'grandma', 
                      'chocolate'] ) # from Ronja's kaggle example data

n_sequences = 30 # 30 videos per sequence / word / sign (get it later from input file?)
sequence_length = 30 # each video with 30 frames (get it later from input file?)
n_keypoints = 1086 # kaggle: 1086; tutorial: 1662

label_map = {label:num for num, label in enumerate(actions)} # create label map (dict, later our .json file)


### Load X and y Data

In [66]:
#loading our preprocessed kaggle data
X = np.load(os.path.join(DATA_PATH, X_fname))
y = np.load(os.path.join(DATA_PATH, y_fname))

### Splitting Train and Test Data

In [67]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

## Build and Train LSTM Neural Network

### Setup Callbacks

### Model building

In [51]:
# setup sequential model
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(n_sequences, n_keypoints))) # input_shape=(#sequences, #keypoints)
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax')) # actions.shape[0]: from user-defined actions (see above)

# compile the model
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
# categorical_crossentropy must be used for multiclass classification model! 

# show summary of model
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_12 (LSTM)              (None, 30, 64)            294656    
                                                                 
 lstm_13 (LSTM)              (None, 30, 128)           98816     
                                                                 
 lstm_14 (LSTM)              (None, 64)                49408     
                                                                 
 dense_12 (Dense)            (None, 64)                4160      
                                                                 
 dense_13 (Dense)            (None, 32)                2080      
                                                                 
 dense_14 (Dense)            (None, 10)                330       
                                                                 
Total params: 449,450
Trainable params: 449,450
Non-tr

### Model fitting

In [52]:
# save model weight for every 100 epochs
model.save_weights(checkpoint_path.format(epoch=10))

model.fit(X_train, y_train, 
          epochs=2000, 
          callbacks=[tb_callback, cp_callback])
# advantage of using mediapipe holistic model is you don't need additional data generator to build up a pipeline of data. Training data fits all into memory.
# 2023-04-07-14-48 run-time: 17m 18.7s

# model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback, cp_callback], validation_data = (x_test, y_test), verbose=0)




Epoch 1/2000
10/10 [==============================] - 3s 35ms/step - loss: 4.7145 - categorical_accuracy: 0.0730
Epoch 2/2000
10/10 [==============================] - 0s 36ms/step - loss: 2.2949 - categorical_accuracy: 0.1111
Epoch 3/2000
10/10 [==============================] - 0s 36ms/step - loss: 2.2906 - categorical_accuracy: 0.1079
Epoch 4/2000
10/10 [==============================] - 0s 39ms/step - loss: 2.2829 - categorical_accuracy: 0.1048
Epoch 5/2000
10/10 [==============================] - 0s 37ms/step - loss: 2.2856 - categorical_accuracy: 0.0952
Epoch 6/2000
10/10 [==============================] - 1s 69ms/step - loss: 2.2684 - categorical_accuracy: 0.1206
Epoch 7/2000
10/10 [==============================] - 0s 43ms/step - loss: 2.2655 - categorical_accuracy: 0.1238
Epoch 8/2000
10/10 [==============================] - 0s 43ms/step - loss: 2.2578 - categorical_accuracy: 0.1429
Epoch 9/2000
10/10 [==============================] - 0s 41ms/step - loss: 2.2563 - categorical_

KeyboardInterrupt: 

### Tensorboard

Follow these steps to look at the model training (also possible in real-time): 

1. Open the command palette (command+shift+P) in VSCode. 
2. Type in "Python: Launch TensorBoard". 
3. Press "Enter". 
4. Click "Use current working directory" (select another option, if you want to look at logs outside the working directory)
5. If "No dashboards are active for the current data set.", click on the circular arrow in the top right to refresh the dashboard. 
6. In the main dashboard area, you should immediately see plots showing the development of accuracies and losses per epoch. 
7. In the left area you can check and uncheck log files that you want to add or remove from the plots. 

## Prediction

### Optional: Load Checkpoint Model

In [ ]:
#checkpoint_path = 'Modeling_logs/Easter_model_10signs_j_test/cp-0800.ckpt'
#model.load_weights(checkpoint_path)

### Predict on test data

In [79]:
y_pred = model.predict(X_test)

1/1 [==============================] - 0s 17ms/step


### Looking at a single example prediction

In [73]:
actions[np.argmax(y_pred[0])] # prediction

'sleepy'

In [85]:
actions[np.argmax(y_test[0])] # actual

'alligator'

## Evaluation using Confusion Matrix and Accuracy

### Back-converting one-hot codes to labels

In [68]:
# convert one-hot encoded categories back to labels, 
# e.g. 0, 1 and 2 instead of [1,0,0], [0,1,0], [0,0,1]
y_test_original = y_test.copy()
y_test = np.argmax(y_test, axis=1).tolist()

In [82]:
y_pred_original = y_pred.copy()
y_pred = np.argmax(y_pred, axis=1).tolist()

### Multilabel Confusion Matrix

In [83]:
# multilabel confusion matrix 
multilabel_confusion_matrix(y_test, y_pred)

array([[[14,  0],
        [ 0,  3]],

       [[13,  0],
        [ 0,  4]],

       [[15,  0],
        [ 0,  2]],

       [[16,  0],
        [ 0,  1]],

       [[16,  0],
        [ 0,  1]],

       [[16,  0],
        [ 0,  1]],

       [[13,  0],
        [ 0,  4]],

       [[16,  0],
        [ 0,  1]]])

### Accuracy Score

In [84]:
# accuracy score
accuracy_score(y_test, y_pred)

1.0

## Save Model Weights

In [ ]:
# save model
model.save(model_name+'.h5')

## Next Steps

Add MODE = 'training' or 'evaluation' or split the notebook into a "Model-Training.ipynb" and "Model-Evaluation.ipynb". 